
# 🚀 BLIGA Demo Notebook

Este notebook muestra paso a paso la implementación del algoritmo **BLIGA** 
(*Best List of Items using Genetic Algorithm*) siguiendo la descripción del artículo.

Cada sección corresponde a las fases vistas en el paper:

1. Dataset de ejemplo
2. Población inicial
3. Correlación (nivel 1 de filtrado)
4. Selección por correlación (topX)
5. Similitud (nivel 2 de filtrado)
6. Operadores genéticos (crossover + mutación)
7. Predicción con Resnick (nivel 3 de filtrado)
8. Selección del mejor individuo


In [4]:

import random
from dataset_example import build_example_dataset
from population import generate_initial_population, select_top_by_correlation
from correlation import correlation_of_individual
from similarity import similarity_of_individual
from operators import one_point_crossover, mutate
from prediction import predict_individual



## 1️⃣ Dataset de ejemplo

Creamos un dataset pequeño de prueba con usuarios, ítems y categorías.


In [5]:

data = build_example_dataset()
print("Usuarios y ratings:", data.ratings)
print("Categorías de ítems:", data.item_categories)


Usuarios y ratings: {'u1': {'I1': 5, 'I2': 4}, 'u2': {'I2': 3, 'I3': 5}, 'u3': {'I1': 2, 'I4': 4, 'I5': 3}}
Categorías de ítems: {'I1': {'acción', 'ciencia_ficción'}, 'I2': {'acción'}, 'I3': {'romance'}, 'I4': {'ciencia_ficción', 'aventura'}, 'I5': {'drama'}, 'I6': {'acción', 'aventura'}}



## 2️⃣ Población inicial

Generamos una población de individuos (listas de ítems no calificados por el usuario activo).


In [6]:

rng = random.Random(42)
population = generate_initial_population(data, target_user="u1", M=6, N=3, rng=rng)
population


[['I4', 'I3', 'I5'],
 ['I5', 'I4', 'I6'],
 ['I4', 'I6', 'I3'],
 ['I3', 'I4', 'I6'],
 ['I4', 'I3', 'I6'],
 ['I4', 'I6', 'I3']]


## 3️⃣ Calcular correlación de cada individuo

Primer nivel de filtrado: **correlación semántica entre ítems** usando Jaccard.


In [7]:

[(ind, correlation_of_individual(ind, data.item_categories)) for ind in population]


[(['I4', 'I3', 'I5'], 0.0),
 (['I5', 'I4', 'I6'], 0.3333333333333333),
 (['I4', 'I6', 'I3'], 0.3333333333333333),
 (['I3', 'I4', 'I6'], 0.3333333333333333),
 (['I4', 'I3', 'I6'], 0.3333333333333333),
 (['I4', 'I6', 'I3'], 0.3333333333333333)]


## 4️⃣ Selección topX por correlación

Nos quedamos con los individuos con mayor coherencia interna de categorías.


In [8]:

best_corr = select_top_by_correlation(population, data, topX=0.5)
best_corr


[['I5', 'I4', 'I6'], ['I4', 'I6', 'I3'], ['I3', 'I4', 'I6']]


## 5️⃣ Similaridad de un individuo

Segundo nivel de filtrado: **similitud usuario-usuario** (Pearson × Jaccard).


In [9]:

for ind in best_corr:
    sim = similarity_of_individual(ind, data, target_user="u1")
    print(ind, "-> similarity =", sim)


['I5', 'I4', 'I6'] -> similarity = 0.0
['I4', 'I6', 'I3'] -> similarity = 0.0
['I3', 'I4', 'I6'] -> similarity = 0.0



## 6️⃣ Operadores genéticos

Aplicamos **crossover** y **mutación** para generar diversidad en la población.


In [25]:

child = one_point_crossover(best_corr[0], best_corr[1], rng)
child = mutate(child, list(data.item_categories.keys()), mutP=0.5, rng=rng)
child


['I5', 'I4', 'I2']


## 7️⃣ Predicción con Resnick

Tercer nivel de filtrado: calculamos la predicción de cada ítem del individuo y 
sumamos para obtener el fitness final del individuo.


In [34]:

scored = [(ind, predict_individual(ind, "u1", data)) for ind in population]
scored.sort(key=lambda t: t[1], reverse=True)
[(ind) for ind in scored]


[(['I4', 'I3', 'I5'], 13.5),
 (['I5', 'I4', 'I6'], 9.0),
 (['I4', 'I6', 'I3'], 9.0),
 (['I3', 'I4', 'I6'], 9.0),
 (['I4', 'I3', 'I6'], 9.0),
 (['I4', 'I6', 'I3'], 9.0)]


## 8️⃣ Selección del mejor individuo

El individuo con mayor predicción total es el que se recomienda al usuario activo.


In [35]:
    best = scored[0]
    print("\nMejor individuo final:", best)


Mejor individuo final: (['I4', 'I3', 'I5'], 13.5)
